In [1]:
import pickle
import torch
from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
# model parameters
head = feedforwardHead_Update
data = '../Data/{}_data_ACSF_expand_PCA_otherInfo.pickle'
batch_size = 32
dim = 128
epochs = 50
clip = 0.4
layer1 = 3
layer2 = 3
factor = 2
lr = 1e-4

In [ ]:
train_dl,val_dl = get_data(data,batch_size)

model = GNN_MataLayer(head,head_mol2,head_atom,head_edge,\
                        dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')
paras = trainable_parameter(model)
opt = Adam(paras,lr=lr)
scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5)

model,train_loss_list,val_loss_list,bestWeight = train_type_earlyStop(opt,model,epochs,train_dl,val_dl,paras,clip,\
                                                                      scheduler=scheduler,weight=0.06)

In [ ]:
save_results(train_loss_list,val_loss_list,reuse,block,\
             head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,postStr='_interleave')
save_model_type(bestWeight,opt,reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,postStr='_interleave')